In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy

from vehicle_lib.vehicle_lib import *

In [2]:

#
# Demo: a vehicle controlled to follow a given path
#
#       Implemented using the code generator openrtdynamics 2 - https://pypi.org/project/openrtdynamics2/ .
#       This generates c++ code for Web Assembly to be run within the browser.
#

system = dy.enter_system()

velocity               = dy.system_input( dy.DataTypeFloat64(1), name='velocity',              default_value=5.0,    value_range=[0, 25],     title="vehicle velocity [m/s]")
modulator              = dy.system_input( dy.DataTypeFloat64(1), name='modulator',             default_value=16.0,   value_range=[0, 100],    title="modulator")
time_scale             = dy.system_input( dy.DataTypeFloat64(1), name='time_scale',            default_value=5.0,    value_range=[0, 6.0],    title="time_scale")
initial_steering       = dy.system_input( dy.DataTypeFloat64(1), name='initial_steering',      default_value=-0.0,   value_range=[-40, 40],   title="initial steering angle [degrees]")       * dy.float64(math.pi / 180.0)
initial_orientation    = dy.system_input( dy.DataTypeFloat64(1), name='initial_orientation',   default_value=0.0,    value_range=[-360, 360], title="initial orientation angle [degrees]")    * dy.float64(math.pi / 180.0)

# parameters
wheelbase = 3.0

# sampling time
Ts = 0.01

# generate a seqeuence for the steering rate
time_instance_indices =            [      50, 100, 150, 250, 300, 350, 400, 450, 500  ]
values                = np.deg2rad([ 0, -1.0,   0, 1.0,  0, -1.0, 0,   0.2, -0.2, 0  ])

steering_rate = dy.signal_step_wise_sequence( time_instance_indices, values, time_scale ) * modulator

# linearly increasing steering angle
delta = dy.euler_integrator( steering_rate, Ts, initial_state=initial_steering )
delta = dy.saturate(u=delta, lower_limit=-math.pi/2.0, upper_limit=math.pi/2.0)

# the model of the vehicle
x, y, psi, x_dot, y_dot, psi_dot = discrete_time_bicycle_model(delta, velocity, Ts, wheelbase, psi0=initial_orientation)


#
# outputs: these are available for visualization in the html set-up
#

dy.append_output(x, 'x')
dy.append_output(y, 'y')
dy.append_output(psi, 'psi')
dy.append_output(delta, 'steering')

# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_template = dy.TargetWasm()
code_gen_results = dy.generate_code(template=code_gen_template, folder="generated/bicycle_open_loop_steering", build=True)

# sourcecode, manifest = dy.generate_code(template=dy.PutBasicRuntimeCpp(i_max=100), folder="generated/bicycle_open_loop_steering2")

algorithm_sourcecode = code_gen_template.get_algorithm_code()

#
dy.clear()


propagating datatypes...
compiling system simulation (level 0)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
Generated code will be written to generated/bicycle_open_loop_steering .
Running compiler: emcc --bind -s MODULARIZE=1 -s EXPORT_NAME="ORTD_simulator" generated/bicycle_open_loop_steering/main.cpp -g4 -s -o generated/bicycle_open_loop_steering/main.js
Compilation result:  32512


In [3]:
import IPython
IPython.display.IFrame(src='../vehicle_control_tutorial/bicycle_open_loop_steering.html', width='100%', height=1000)